In [17]:
from IPython.display import Latex

In [18]:
%%javascript
MathJax.Extension["TeX/cancel"] = {
    version: "2.4.0",
    ALLOWED: {
        color: 1,
        mathcolor: 1,
        background: 1,
        mathbackground: 1,
        padding: 1,
        thickness: 1
    }
};
MathJax.Hub.Register.StartupHook("TeX Jax Ready", function () {
    var c = MathJax.InputJax.TeX,
        a = MathJax.ElementJax.mml,
        b = MathJax.Extension["TeX/cancel"];
    b.setAttributes = function (h, e) {
        if (e !== "") {
            e = e.replace(/ /g, "").split(/,/);
            for (var g = 0, d = e.length; g < d; g++) {
                var f = e[g].split(/[:=]/);
                if (b.ALLOWED[f[0]]) {
                    if (f[1] === "true") {
                        f[1] = true
                    }
                    if (f[1] === "false") {
                        f[1] = false
                    }
                    h[f[0]] = f[1]
                }
            }
        }
        return h
    };
    c.Definitions.Add({
        macros: {
            cancel: ["Cancel", a.NOTATION.UPDIAGONALSTRIKE],
            bcancel: ["Cancel", a.NOTATION.DOWNDIAGONALSTRIKE],
            xcancel: ["Cancel", a.NOTATION.UPDIAGONALSTRIKE + " " + a.NOTATION.DOWNDIAGONALSTRIKE],
            cancelto: "CancelTo"
        }
    }, null, true);
    c.Parse.Augment({
        Cancel: function (e, g) {
            var d = this.GetBrackets(e, ""),
                f = this.ParseArg(e);
            var h = b.setAttributes({
                notation: g
            }, d);
            this.Push(a.menclose(f).With(h))
        },
        CancelTo: function (e, g) {
            var i = this.ParseArg(e),
                d = this.GetBrackets(e, ""),
                f = this.ParseArg(e);
            var h = b.setAttributes({
                notation: a.NOTATION.UPDIAGONALSTRIKE + " " + a.NOTATION.UPDIAGONALARROW
            }, d);
            i = a.mpadded(i).With({
                depth: "-.1em",
                height: "+.1em",
                voffset: ".1em"
            });
            this.Push(a.msup(a.menclose(f).With(h), i))
        }
    });
    MathJax.Hub.Startup.signal.Post("TeX cancel Ready")
});
MathJax.Ajax.loadComplete("[MathJax]/extensions/TeX/cancel.js");

<IPython.core.display.Javascript object>

$$
\newcommand{\hyperlink}[1]{}
\newcommand{\linkdest}[1]{}
\newcommand{\hypertarget}[1]{}
\newcommand{\hcancel}[1]{%
    \cancel{\mathrm{#1}}%
}%
$$

# Ethereum Yellow Paper超ふわっと要約

2018-12-10版Yellow Paperを要約しつつ重要なことを書き足す。進捗 : 7/39ページ

## 2 ブロックチェーンの定式化

Yellow Paperのセクション２では、すべての脱中央集権的ブロックチェーンに共通の構造を定式化している。
ブロックチェーンとは何かしらの内部状態を保持するデータベースであり、トランザクションとブロックの二重構造により内部状態（ステート）を変化させる。

- トランザクションによるステートの変更 - トランザクション$T$が$\Upsilon$関数を用いてステート$\sigma_t$を更新する。
$$ \sigma_{t+1} \equiv \Upsilon(\sigma_t, T)$$

- ブロックによるステートの変更 - ブロック$B$が$\Pi$関数を用いてステート$\sigma_t$を更新する。$B$はトランザクションの列とその他ブロック固有のパラメータで構成されている。$\Pi$はステートをブロック内の全トランザクションの分更新させたのち、マイニング報酬を与えるなどの最終処理関数$\Omega$を用いてステートを更新する。
$$ \sigma_{t+1} \equiv \Pi(\sigma_t, B) $$
$$ B \equiv (\dots, (T_0, T_1, \dots), \dots) $$
$$ \Pi(\sigma, B) \equiv \Omega(B, \dots\Upsilon(\Upsilon(\sigma, T_0), T_1)\dots) $$

Ethereumにおけるこれらのステート更新関数の具体的な定義はまた後ほど紹介する。

## 2.1 単位
続いてセクション２では通過単位を定義している。最小単位はWeiであり、それを元に次の表のように倍率を設定している。

|Multiplier|Name   |
|----------|-------|
|$10^0$      |Wei    |
|$10^{12}$     |Szabo  |
|$10^{15}$     |Finney |
|$10^{18}$     |Ether  |



In [22]:
# An abstruct definition of a Blockchain.
class Blockchain:
    def __init__(self, init_state, tx_update, block_finalize):
        # input: state and tx
        # output: state
        self.tx_update = tx_update
        
        # input: block and state
        # output: state
        self.block_finalize = block_finalize
        
        # input: state and block
        # output: state
        self.block_update = lambda s, b: block_finalize(b, reduce(tx_update, b.transactions, s))

# ↓↓↓Start actual definition of an Ethereum Blockchain↓↓↓

wei = 1
szabo = 10**12
finney = 10**15
ether = 10**18

## 4.1 ステートの定義

ここではEthereumにおけるステートの定義を話す。

### 構成

Ethereumにおけるブロックチェーン全体のステート(ワールドステート、前項における$\sigma$)とは、アドレスとアカウントステートとの間の写像のこと。つまり、あるアドレスが与えられると、そのアドレスの状態（アカウントステート）を取得できるテーブルのことである。ここでアドレスは160ビットの整数であり、アカウントステートはRLPでエンコードされたデータである。

RLPとはデータ配列をバイト列に変換する関数であり、$\mathtt{RLP}$と表記する。

EthereumではハッシュにKeccak-256関数($\mathtt{KEC}$と表記)が用いられる。

ワールドステートはtrieという独自のハッシュ木で構成される。trieはハッシュ化することができ、keyとvalueの組$\mathfrak{I} = \{(k_0, v_0), (k_1, v_1), \dots\}$に対するハッシュを$\mathtt{TRIE}(\mathfrak{I})$と表す。

$a$をアカウントとした時のアカウントステート$\sigma[a]$は以下の4つで構成される。
- nonce: アカウントが今までに発行したトランザクション数。$\sigma[a]_n$と表記。
- balance: アカウントの残高。$\sigma[a]_b$と表記。
- storageRoot: アカウントの持つストレージのハッシュ。ストレージは256ビットの値同士の組を保持することができるもので、ワールドステートと同様にtrieである。$\sigma[a]_\mathrm{s}$と表記。対応するストレージそのものは$\sigma[a]_\mathbf{s}$と表記。
- codeHash: アカウントの持つプログラムコードのハッシュ。$\sigma[a]_c$と表記。コードそのものを$b$と表記することがあり、その時は$\sigma[a]_c = \mathtt{KEC}(b)$である。

### データの処理

以下のように、ストレージのハッシュ$\sigma[a]_s$は、ストレージのもつ全てのキー$k$とバリュー$v$の組について、$\mathtt{KEC}(k)$と$\mathtt{RLP}(v)$を順々に並べていったものを$\mathtt{TRIE}$でハッシュしたものである。ここで、$L_{I}$は$k$と$v$を適切な形にエンコードする関数である。

また、関数$f(x)$に対し$f^*(x)$を以下で定義する。$*$はいわゆる配列のmap関数である。

$$
f^*\big( (x_0, x_1, ...) \big) \equiv \big( f(x_0), f(x_1), ... \big) \quad \text{for any function} \; f
$$

$$ \mathtt{TRIE}(L_I^*(\sigma[a]_\mathbf{s})) \equiv \sigma[a]_\mathrm{s}$$

$$ L_{I}\big( (k, v) \big) \equiv \big(\texttt{KEC}(k), \texttt{RLP}(v)\big) $$

$\sigma[a]_c = \mathtt{KEC}(())$の時、すなわちアカウントのプログラムが空であるとき、そのアカウントは「普通の」アカウント、コントラクトでないアカウントである。

ワールドステートのエンコード関数$L_S$を定義する。これはワールドステートから存在するアカウントのみを抽出し、配列に変換する関数である。
$$L_S(\sigma) \equiv \{p(a) : σ[a] \neq \emptyset \}$$
ここで、
$$p(a) \equiv (\mathtt{KEC}(a), \mathtt{RLP}((\sigma[a]_n, \sigma[a]_b, \sigma[a]_s, \sigma[a]_c)))$$
である。

ワールドステートのハッシュの際には、$L_S$と$\mathtt{TRIE}$を用いてハッシュする。以降、ワールドステートは次の形式に沿っているものとする。

1. ワールドステートに含まれる全てのアカウントは存在しないか、もしくはアドレス(trieのキー)が20バイトでそのアカウントステートが正しい形式かである。
$$\forall a : \sigma[a] = \emptyset \lor (a \in \mathbb{B}_{20} \land v(\sigma[a]))$$
2. アカウントステートが正しい形式であるとは、nonceが256ビット整数、balanceが256ビット整数、storageRootが32バイト列、codeHashが32バイト列であることである。
$$v(x) \equiv x_n \in \mathbb{N}_{256} \land x_b \in \mathbb{N}_{256} \land x_s \in \mathbb{B}_{32} \land x_c \in \mathbb{B}_{32}$$

### アカウントの空・死

アカウント$a$が空であるとは、そのコードが空であり、そのnonceが0であり、その残高が0であることである。
$$\mathtt{EMPTY}(\sigma,a) \equiv \sigma[a]_c = \mathtt{KEC}(()) \land \sigma[a]_n = 0 \land \sigma[a]_b = 0$$

アカウント$a$が死んでいるとは、そのアカウントが存在しないか、空であることである。
$$\mathtt{DEAD}(\sigma,a) \equiv \sigma[a] = \emptyset \lor \mathtt{EMPTY}(\sigma,a)$$



In [31]:
def encode_keyvalue(tpl):
    (k, v) = tpl
    return (KEC(k), RLP(v))

def encode_account(address, state):
    return (KEC(address), RLP(tuple(state[address])))

def encode_state(s):
    return map(encode_account, s.items())

from collections import namedtuple
State = dict
Address = namedtuple("Address", "nonce balance storageRoot codeHash")

def EMPTY(s, a):
    return s[a].codeHash == KEC(()) and s[a].nonce == 0 and s[a].balance == 0

def DEAD(s, a): 
    return (a not in s) or EMPTY(s, a)

## 4.2 トランザクションの定義

ここではトランザクションの定義を話す。

### 構成

トランザクションとはEthereumの外部から投稿される、署名された（ステートを更新させる）命令のことである。これは以下の要素からなる。

- nonce : トランザクションを送信したアカウントが今までに発行したトランザクション数。$T_n$と表記。
- gasPrice : 1gasあたりにかかるweiの量を指定する。$T_p$と表記。
- gasLimit : 計算に最大で何gas使えるかを指定する。この値は前払いであり、計算が行われる前から引き落とされる。$T_g$と表記。
- to : 送金先アドレス。160ビット数値。$T_t$と表記。
- value : アドレスtoに送金されるWeiの量。$T_v$と表記。
- v, r, s : トランザクションのECDSA署名。これを用いてトランザクションを作成したアカウント(sender)を計算する。$T_w, T_r, T_s$と表記。

コントラクト作成トランザクションの場合、以下の要素がある。

- init : コントラクトが作成されるときに使われるEVMコードを指定する。可変長バイト列。$T_i$と表記。

initを実行すると、bodyと呼ばれるコードが返される。bodyはコントラクトを呼ぶたびに実行されるコードで、コントラクトへの入力を処理する。つまり、initは「bodyというEVMプログラムを生成するEVMプログラム」ということになる。

コントラクトを呼び出すトランザクションの場合、以下の要素がある。

- data : コントラクトに入力するデータを指定する。可変長バイト列。$T_d$と表記。

### データの処理

実際にトランザクションを処理するために必要な関数を定義していく。

$S(T)$はトランザクションからそのトランザクションを作成したアカウントを計算する。$T_w, T_r, T_s$を使うと、ECDSAの性質から公開鍵を算出できる。

$L_T(T)$は$T_t$の有無からそのトランザクションがコントラクト呼び出し命令なのか作成命令なのかを判別し、適切な配列に並べる関数である。

$$
L_T(T)
\equiv
\left
\{
\begin{array}{l}
(T_n,T_p,T_g,T_t,T_v,T_\mathbf{i},T_w,T_r,T_s) & \text{if} \quad T_t = \emptyset \\
(T_n,T_p,T_g,T_t,T_v,T_\mathbf{d},T_w,T_r,T_s) & \text{otherwise}
\end{array}
\right.
$$

以降、

- nonce, gasPrice, gasLimit, value, r, sは256ビット数値
- vは5ビット数値
- data, initは可変長バイト列
- toは20バイト列　もしくは　空文字列

とする。toを空文字列にした場合、トランザクションはコントラクト作成命令になる。

$$
\begin{array}[t]{lclclc}
T_{\mathrm{n}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{v}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{p}} \in \mathbb{N}_{256} & \wedge \\
T_{\mathrm{g}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{w}} \in \mathbb{N}_5 & \wedge & T_{\mathrm{r}} \in \mathbb{N}_{256} & \wedge \\
T_{\mathrm{s}} \in \mathbb{N}_{256} & \wedge & T_{\mathbf{d}} \in \mathbb{B} & \wedge & T_{\mathbf{i}} \in \mathbb{B}
\end{array}
$$

$$
T_{\mathbf{t}} \in \begin{cases} \mathbb{B}_{20} & \text{if} \quad T_{\mathrm{t}} \neq \varnothing \\
\mathbb{B}_{0} & \text{otherwise}\end{cases}
$$

In [34]:
def encode_transaction(t):
    if t.to == "":
        return tuple(t.nonce, t.gasPrice, t.gasLimit, t.to, t.value, t.init, t.v, t.r, t.s)
    else:
        return tuple(t.nonce, t.gasPrice, t.gasLimit, t.to, t.value, t.data, t.v, t.r, t.s)

Transaction = namedtuple("Transaction", "nonce gasPrice gasLimit to value v r s init data")

## 4.3 ブロックの定義

ここではブロックの定義について話す。

### 構成

Ethereumではブロックタイムを短縮するため、一本鎖に繋がれたブロック以外にも報酬が行き渡るようになっている。つまり、ある程度はフォークのブロックにもマイニング報酬が支払われるということだ。

ちなみに2019/3/2現在、EthereumではまだPoSは実装されていなく、完全にPoWである。このことは次に紹介するブロック構造を見てもよくわかる。

ブロックヘッダは以下で構成される。

- parentHash: 親のブロックのハッシュ。$H_{\mathrm{p}}$と表記。
- ommersHash: 「『自分のブロックとおなじ親の親』を親に持つブロックたち」（ommers）を総合したハッシュ。$H_{\mathrm{o}}$と表記。
- beneficiary: マイニング報酬を受け取るアドレス。$H_{\mathrm{c}}$と表記。
- stateRoot: ワールドステートのハッシュ。前述の通りワールドステートはアカウントとアカウントステートのtrie。$H_{\mathrm{r}}$と表記。
- transactionsRoot: 全トランザクションのハッシュ。リストをtrieにしてハッシュする。$H_{\mathrm{t}}$と表記。
- receiptsRoot: 全レシートのハッシュ。リストをtrieにしてハッシュする。$H_{\mathrm{e}}$と表記。
- logsBloom: 全ログのハッシュ。リストをtrieにしてハッシュする。$H_{\mathrm{b}}$と表記。
- difficulty: ブロックのマイニング難易度。$H_{\mathrm{d}}$と表記。
- number: ブロック番号。$H_{\mathrm{i}}$と表記。
- gasLimit: ブロック全体の最大gas消費量（最大計算量）。$H_{\mathrm{l}}$と表記。
- gasUsed: ブロック全体のgas消費量（計算量）。$H_{\mathrm{g}}$と表記。
- timestamp: ブロックのタイムスタンプ。$H_{\mathrm{s}}$と表記。
- extraData: 32バイト以内の恣意的な文字列。$H_{\mathrm{x}}$と表記。
- mixHash, nonce: 所定の量の計算量を消費（マイニング）したことを証明する。mixHash=$H_{\mathrm{m}}$、nonce=$H_{\mathrm{n}}$と表記。

レシートはトランザクションが実行された証明のことである。一つのトランザクションにつき一つ生成される。

ログはトランザクションやアカウントの検索を行うためのマーキングデータである。コントラクト内で発生させることができる。（ログを記録するEVM命令が存在する。）

ommersは人間で言うところの"おじ"や"おば"に当たる。

ブロック$B$はブロックヘッダ$B_H$、トランザクションのリスト$B_T$、ommerのヘッダのリスト$B_U$からなる。レシートやログはトランザクションから計算されるため、この３つで十分である。
$$B \equiv (B_H,B_T,B_U)$$


In [15]:
BlockHeader = namedtuple("BlockHeader", "parentHash ommersHash beneficiary stateRoot transactionsRoot receiptsRoot logsBloom difficulty number gasLimit gasUsed timestamp extraData mixHash nonce")

Block = namedtuple("Block", "blockheader transactions ommers")

### 4.3.1 レシートとログ
レシート$R$は各トランザクションから算出される、支払い証明や検索のために使うデータのことである。これは以下の要素からなる。

- $R_\mathrm{u}$ : そのトランザクションが所属するブロックにおいて、そのトランザクションが処理された直後までに使用されたgasの累計。
- $R_\mathrm{l}$ : そのトランザクションが実行される時に生成された全てのログ。
- $R_\mathrm{b}$ : そのログから生成されたBloomフィルター。
- $R_\mathrm{z}$ : トランザクションのステータスコード。

$$R \equiv (R_\mathrm{u}, R_\mathrm{b}, R_\mathrm{l}, R_\mathrm{z})$$

今後のデータ処理のため、以下の関数を定義する。最初に0で埋まった256バイト列を追加したのは、以前のプロトコルに存在したpre-transaction stateというパラメータを置き換えるためである。

$$L_R(R) \equiv (0 \in \mathbb{B}_{256}, R_\mathrm{u}, R_\mathrm{b}, R_\mathrm{l})$$

ステータスコード$ R_\mathrm{z}$は負でない整数とする。

$$ R_\mathrm{z} \in \mathbb{N}$$

gas累計$ R_\mathrm{u} $は正の整数であり、Bloomフィルター$ R_\mathrm{b} $は256バイト列とする。

$$ R_\mathrm{u} \in \mathbb{N} \land R_\mathrm{b} \in \mathbb{B}_{256}$$

$R_\mathrm{l}$はログの配列である。

$$R_\mathrm{l} = (O_0, O_1, ...)$$

ログ$O$はコントラクト内で発生させることができる、トランザクションやアカウントの検索を行うためのマーキングデータである。これは以下の要素からなる。

- $O_\mathrm{a}$ : ログ発生者のアドレス
- $O_\mathbf{t}$（トピック）: 32バイト列のリスト
- $O_\mathbf{d}$ : 可変長のバイト列

$$
O \equiv (O_{\mathrm{a}}, ({O_{\mathbf{t}}}_0, {O_{\mathbf{t}}}_1, ...), O_{\mathbf{d}})
$$

$$
O_{\mathrm{a}} \in \mathbb{B}_{20} \quad \wedge \quad \forall x \in O_{\mathbf{t}}: x \in \mathbb{B}_{32} \quad \wedge \quad O_{\mathbf{d}} \in \mathbb{B}
$$

Bloomフィルター関数$M$を定義する。これは一つのログを256バイトにハッシュ化するものである。ただし、その際に$O_\mathbf{d}$は無視される。まず、$O$のアドレス$O_\mathrm{a}$と全てのトピック${O_\mathbf{t}}_0, {O_\mathbf{t}}_1, ...$をそれぞれ「3つの0~2047までの値」にし、その値が指すビットを全て1にした256バイトの値を返す。

$\mathbf{x}$に対する「3つの0~2047までの値」は、$\mathtt{KEC}(\mathbf{x})$の0~1バイト目、2~3バイト目、4~5バイト目をそれぞれ2048で割ったあまりと定義する。

式にするとこうなる。ここでは$\bigvee$はビットORを表す。

$$
M(O) \equiv \bigvee_{x \in \{O_{\mathrm{a}}\} \cup O_{\mathbf{t}}} \big( M_{3:2048}(x) \big)
$$

$$
\begin{array}{rcl}
M_{3:2048}(\mathbf{x}: \mathbf{x} \in \mathbb{B}) & \equiv & \mathbf{y}: \mathbf{y} \in \mathbb{B}_{256} \quad \text{where:}\\
\mathbf{y} & = & (0, 0, ..., 0) \quad \text{except:}\\
\forall_{i \in \{0, 2, 4\}}&:& \mathcal{B}_{m(\mathbf{x}, i)}(\mathbf{y}) = 1\\
m(\mathbf{x}, i) &\equiv& \mathtt{KEC}(\mathbf{x})[i, i + 1] \bmod 2048
\end{array}
$$

ここで$\mathcal{B}_j(\mathbf{x})$は$\mathbf{x}$の$j$ビット目を表す。

Bloomフィルターとはその名の通りフィルターであり、検索のために使うデータである。例えばあるトピックを持つ（可能性がある）トランザクションを全て列挙したい時、そのトピックに対する$M_{3:2048}(x)$関数を呼ぶと、2048ビットの中で3つの箇所だけが1になっている数が出力される。その数と同じ箇所が1になっているBloomフィルターを持つトランザクションに検索したいトピックがある可能性が高い。そうやって大まかな絞り込みをした後に、実際にログに検索をかければ良いということになる。

式ではわかりにくいので以下にコードを示す。

In [5]:
def bloom_filter(log):
    address = log.address
    topics = log.topics
    out = 0
    
    # アドレスと全トピックに対して...
    for x in [address] + topics:
        # ハッシュを計算
        h = KEC(x)
        
        # 計算したハッシュの0~1  2~3   4~5バイト目を読み取る
        for i in [0, 2, 4]:
            # ビットを1にする位置を計算
            m = (ord(h[i])*256 + ord(h[i+1])) % 2048
            
            # 計算したビットを1にする
            out = out | (2**m)
    
    # 数値を256バイトに変換
    return out.to_bytes(256, "big")

Receipt = namedtuple("Receipt", "gas logs bloom status")
Log = namedtuple("Log", "address topics data")

encode_receipt = lambda r: tuple("\00" * 256, r.gas, tuple(r.logs), r.bloom)

### 4.3.2 ブロックヘッダの検証（ハッシュ）

Ethereumノードはブロックに含まれているトランザクション$B_T$やommerのヘッダ$B_U$を使ってステート$\sigma$やそのハッシュなどを計算して、その結果をブロックヘッダと照合する。同時にこれはマイナーがブロックヘッダを計算するときの計算式にもなっている。

各ハッシュは以下で定義される。

- ワールドステートのハッシュ$H_{\mathrm{r}}$ : 今までのステート$\boldsymbol{\sigma}$を$\Pi$で更新し、$L_S$と$\mathtt{TRIE}$でハッシュ化。
- ommersのヘッダのハッシュ$H_{\mathrm{o}}$ : $L_H^*$でommerのヘッダ$B_{\mathbf{U}}$をRLPにしたのち、$\mathtt{KEC}$でハッシュ化。
- 全トランザクションのハッシュ$H_{\mathrm{t}}$ : トランザクション番号$i$とトランザクションの内容をエンコードしたもの$L_{T}(B_{\mathbf{T}}[i])$のペアを$\mathtt{TRIE}$でハッシュ化。
- 全レシートのハッシュ$H_{\mathrm{e}}$ : トランザクション番号$i$とレシートの内容をエンコードしたもの$L_{R}(B_{\mathbf{R}}[i]))$のペアを$\mathtt{TRIE}$でハッシュ化。
- 全ログのハッシュ$H_{\mathrm{b}}$ : 全てのBloomフィルターをビットORする。

$$
\begin{array}[t]{lclc}
H_{\mathrm{r}} &\equiv& \mathtt{TRIE}(L_S(\Pi(\boldsymbol{\sigma}, B))) & \wedge \\
H_{\mathrm{o}} &\equiv& \mathtt{KEC}(\mathtt{RLP}(L_H^*(B_{\mathbf{U}}))) & \wedge \\
H_{\mathrm{t}} &\equiv& \mathtt{TRIE}(\{\forall i < \lVert B_{\mathbf{T}} \rVert, i \in \mathbb{N}: &\\&& \quad\quad p (i, L_{T}(B_{\mathbf{T}}[i]))\}) & \wedge \\
H_{\mathrm{e}} &\equiv& \mathtt{TRIE}(\{\forall i < \lVert B_{\mathbf{R}} \rVert, i \in \mathbb{N}: &\\&& \quad\quad p(i, L_{R}(B_{\mathbf{R}}[i]))\}) & \wedge \\
H_{\mathrm{b}} &\equiv& \bigvee_{\mathbf{r} \in B_{\mathbf{R}}} \big( \mathbf{r}_{\mathrm{b}} \big)
\end{array}
$$

ここで今までにデータ処理用に定義してきた関数が登場する。

- $L_S$ : ワールドステートから存在するアカウントのみを抽出し、$(\mathtt{KEC}(a), \mathtt{RLP}((\sigma[a]_n, \sigma[a]_b, \sigma[a]_s, \sigma[a]_c)))$の形式の配列にエンコードする。
- $L_T$ : トランザクションがコントラクト呼び出し命令なのか作成命令なのかを判別し、適切な配列に並べる。
- $L_R$ : 最初の256バイトは全て0で、$R_\mathrm{u}, R_\mathrm{b}, R_\mathrm{l}$が続く。（$R_\mathrm{z}$は無視。）

ommersをエンコードする$L_H^*$は次項で定義する。

また、$p$は値のペアをRLP形式に変換する関数である。

$$
p(k, v) \equiv \big( \mathtt{RLP}(k), \mathtt{RLP}(v) \big)
$$


In [9]:
get_state_root = lambda prev_state, block: TRIE(encode_state(block_update(prev_state, block)))
get_ommers_hash = lambda ommers: KEC(RLP(tuple(map(encode_blockheader, ommers))))
get_transactions_root = lambda transactions: TRIE([p(i, encode_transaction(v)) for i, v in enumerate(transactions)])
get_receipts_root = lambda receipts: TRIE([p(i, encode_receipt(v)) for i, v in enumerate(receipts)])
get_logs_bloom = lambda logs: reduce(lambda x, y : x | y, map(lambda l: int.from_bytes(bloom_filter(l), "big"), logs)).to_bytes(256, "big")

### 4.3.3 シリアライゼーション

ブロックヘッダのシリアライズ関数$L_{H}$と、ブロックのシリアライズ関数$L_{B}$を以下で定義する。ヘッダに関しては、単純にパラメータを配列にするだけ。ブロックに関しては、いままで定義してきたデータ処理用の関数を適用して配列にする。

$$
\begin{array}[t]{}
\quad L_{H}(H) & \equiv & (\begin{array}[t]{l}H_{\mathrm{p}}, H_{\mathrm{o}}, H_{\mathrm{c}}, H_{\mathrm{r}}, H_{\mathrm{t}}, H_{\mathrm{e}}, H_{\mathrm{b}}, H_{\mathrm{d}},\\ H_{\mathrm{i}}, H_{\mathrm{l}}, H_{\mathrm{g}}, H_{\mathrm{s}}, H_{\mathrm{x}}, H_{\mathrm{m}}, H_{\mathrm{n}} \; )\end{array} \\
\quad L_{B}(B) & \equiv & \big( L_{H}(B_{H}), L_{T}^*(B_{\mathbf{T}}), L_{H}^*(B_{\mathbf{U}}) \big)
\end{array}
$$

先ほどommerのヘッダの配列に対して$L_H^*$を使ったが、これはommerのヘッダの配列の要素それぞれをシリアライズした配列を作る関数ということになる。

各パラメータの型は以下の通り。

$$
\begin{array}[t]{lclclcl}
H_{\mathrm{p}} \in \mathbb{B}_{32} & \wedge & H_{\mathrm{o}} \in \mathbb{B}_{32} & \wedge & H_{\mathrm{c}} \in \mathbb{B}_{20} & \wedge \\
H_{\mathrm{r}} \in \mathbb{B}_{32} & \wedge & H_{\mathrm{t}} \in \mathbb{B}_{32} & \wedge & H_{\mathrm{e}} \in \mathbb{B}_{32} & \wedge \\
H_{\mathrm{b}} \in \mathbb{B}_{256} & \wedge & H_{\mathrm{d}} \in \mathbb{N} & \wedge & H_{\mathrm{i}} \in \mathbb{N} & \wedge \\
H_{\mathrm{l}} \in \mathbb{N} & \wedge & H_{\mathrm{g}} \in \mathbb{N} & \wedge & H_{\mathrm{s}} \in \mathbb{N}_{256} & \wedge \\
H_{\mathrm{x}} \in \mathbb{B} & \wedge & H_{\mathrm{m}} \in \mathbb{B}_{32} & \wedge & H_{\mathrm{n}} \in \mathbb{B}_{8}
\end{array}
$$

In [37]:
encode_blockheader = tuple

encode_block = lambda b: (encode_header(b.blockheader), tuple(map(encode_transaction, b.transactions)), tuple(map(encode_blockheader, b.ommers)))

### 4.3.4 ブロックヘッダの検証（その他のパラメータ）

ブロックの検証にはハッシュだけでなく、マイニング難易度などの数値も関係している。

#### ブロックナンバー

ブロックナンバー$H_{\mathrm{i}}$は親のブロックナンバーに1加算したものである。（$P(H)_{H}$は親のブロックを表す。）

$$
H_{\mathrm{i}} \equiv {{P(H)_{H}}_{\mathrm{i}}} + 1
$$

#### 難易度(diifficulty)

有効な難易度$D(H)$は以下で定義される。これはブロックヘッダに書き込まれる難易度$H_{\mathrm{d}}$と等しくなければならない。割り算・指数関数は計算のたびに小数点以下切り捨てとする。

- 最初のブロックの難易度は131072（$D_0$）。
- 最初のブロックでない場合、以下の和を計算する。その値が$D_0$に満たない場合は、$D_0$が難易度になる。
  - ${P(H)_{H}}_{\mathrm{d}}$ : 親のdifficulty
  - $x\times\varsigma_2$ : 以下を掛けたもの
    - $x$ : 親のdifficulty / 2048
    - $\varsigma_2$ : 以下の値と-99との大きい方
      - ommerがない場合1、ommerがある場合2から、
      - (親のブロックとのタイムスタンプの差 / 9)を減じた値。
  - $\epsilon$ : 2の(((ブロック番号 - 3000000) / 100000) - 2)乗。ただしブロック番号が3000000未満の場合は0。

数式で書くと:

$$
D(H) \equiv \begin{cases}
D_0 & \text{if} \quad H_{\mathrm{i}} = 0\\
\text{max}\!\left(D_0, {P(H)_{H}}_{\mathrm{d}} + x\times\varsigma_2 + \epsilon \right) & \text{otherwise}\\
\end{cases}
$$

ただし

$$
D_0 \equiv 131072
$$

$$
x \equiv \left\lfloor\frac{{P(H)_{H}}_{\mathrm{d}}}{2048}\right\rfloor
$$

$$
\varsigma_2 \equiv \text{max}\left( y - \left\lfloor\frac{H_{\mathrm{s}} - {P(H)_{H}}_{\mathrm{s}}}{9}\right\rfloor, -99 \right)
$$

$$
y \equiv \begin{cases}
1 & \text{if} \, \lVert P(H)_{\mathbf{U}}\rVert = 0 \\
2 & \text{otherwise}
\end{cases}
$$

$$
\begin{align}
\epsilon &\equiv \left\lfloor 2^{ \left\lfloor H'_{\mathrm{i}} \div 100000 \right\rfloor - 2 } \right\rfloor \\
H'_{\mathrm{i}} &\equiv \max(H_{\mathrm{i}} - 3000000, 0)
\end{align}
$$

説明すると、

- まず親の難易度を加算する。 
- (親の難易度 / 2048)に特定の係数$\varsigma_2$を掛けたものを加算する。$\varsigma_2$はommerがあると大きくなり、マイニングにかかった時間（前のブロックとのタイムスタンプの差）が大きいほど小さくなる。ただし-100以下にはならない（すなわち親の難易度の約2048分の100=約5%までしか影響しない）。
  - $\varsigma_2$が0になる時、ブロックのタイムスタンプの差はommerがある時は約18秒、ommerがない時は約9秒である。これはつまり（ommerの有無を1ブロックと数えた上での）ブロック生成頻度が約9秒に1回になるように調節するということである。
- $\epsilon$は3000000ブロック以降では指数関数的に難易度が上昇することを意味している。これはPoWからPoSへ移行するためのメカニズムである。
  - 第7334584ブロック現在のメインネットの難易度は1888487258439463で、$\epsilon$の値は2199023255552であるから、今はまだ0.1%ほどの影響力しかないが、指数関数なのでそのうち"爆弾"のようにPoWのシステムを破綻させる。

#### gasLimit

1ブロック内のガス最大消費量であるgasLimitは、以下の条件を満たす必要がある。

$$
\begin{array}[t]{}
& & H_{\mathrm{l}} < {P(H)_{H}}_{\mathrm{l}} + \left\lfloor\frac{{P(H)_{H}}_{\mathrm{l}}}{1024}\right\rfloor \quad \wedge \\
\nonumber& & H_{\mathrm{l}} > {P(H)_{H}}_{\mathrm{l}} - \left\lfloor\frac{{P(H)_{H}}_{\mathrm{l}}}{1024}\right\rfloor \quad \wedge \\
\nonumber& & H_{\mathrm{l}} \geqslant 5000
\end{array}
$$

これはマイナーに対し、親ブロックのgasLimitの約$\pm\frac{1}{1024}$の割合(約0.2%の幅)だけ裁量を任せているという意味になる。

#### ブロックタイム

ブロックタイムは必ず親のブロックタイムより大きくなければならない。
$$
H_{\mathrm{s}} > {P(H)_{H}}_{\mathrm{s}}
$$

#### nonce

mixHashを除いたブロックのパラメータを後述の$\mathtt{PoW}$関数を用いて$n$と$m$という値にする。$n$は2の256乗を難易度で割ったもの以下でなければいけない。$m$はmixHashとなる。

$$
n \leqslant \frac{2^{256}}{H_{\mathrm{d}}} \quad \wedge \quad m = H_{\mathrm{m}}
$$

$$
n, m = \mathtt{PoW}(H_{\hcancel{n}}, H_{\mathrm{n}}, \mathbf{d})
$$

ここで$H_{\hcancel{n}}$はブロックヘッダからnonceを除いたものである。$\mathbf{d}$はDAGと呼ばれるハッシュするのに必要なデータセットのことである。$\mathtt{PoW}$関数にランダムに$H_{\mathrm{n}}$を入力していったとして、上記の不等式を満たす$n$を生成するために必要な計算回数はだいたい$H_{\mathrm{d}}$に比例する。

#### まとめ

結局、以下の命題関数によってブロックが正当なものかどうかが検証されることとなる。

\begin{eqnarray}
V(H) & \equiv &  n \leqslant \frac{2^{256}}{H_{\mathrm{d}}} \wedge m = H_{\mathrm{m}} \quad \wedge \\
\nonumber & & H_{\mathrm{d}} = D(H) \quad \wedge \\
\nonumber& & H_{\mathrm{g}} \le H_{\mathrm{l}}  \quad \wedge \\
\nonumber& & H_{\mathrm{l}} < {P(H)_{H}}_{\mathrm{l}} + \left\lfloor\frac{{P(H)_{H}}_{\mathrm{l}}}{1024}\right\rfloor  \quad \wedge \\
\nonumber& & H_{\mathrm{l}} > {P(H)_{H}}_{\mathrm{l}} - \left\lfloor\frac{{P(H)_{H}}_{\mathrm{l}}}{1024}\right\rfloor  \quad \wedge \\
\nonumber& & H_{\mathrm{l}} \geqslant 5000  \quad \wedge \\
\nonumber& & H_{\mathrm{s}} > {P(H)_{H}}_{\mathrm{s}} \quad \wedge \\
\nonumber& & H_{\mathrm{i}} = {P(H)_{H}}_{\mathrm{i}} +1 \quad \wedge \\
\nonumber& & \lVert H_{\mathrm{x}} \rVert \le 32
\end{eqnarray}

## 6. トランザクション実行

この章ではトランザクションの実行、すなわちEthereumにおける$\Upsilon$関数を定義する。また、

- トランザクションによるトランザクションからgasを計算する$\Upsilon^{\mathrm{g}}$
- トランザクションからログを計算する$\Upsilon^{\mathbf{l}}$
- トランザクションのステータスコード(成功か失敗か)を計算する$\Upsilon^{\mathrm{z}}$

の定義をする。

ここで復習すると、$\Upsilon$とは以下のようにトランザクションを使ってステートを更新する関数である。

$$
\boldsymbol{\sigma}' = \Upsilon(\boldsymbol{\sigma}, T)
$$

### 6.1 副ステート

ステートを更新するためにトランザクションから計算される副ステートと言う値を定義する。それを$A$と表す。

$$
A \equiv (A_{\mathbf{s}}, A_{\mathbf{l}}, A_{\mathbf{t}}, A_{\mathrm{r}})
$$

ここで、

- $A_{\mathbf{s}}$ : 実行時に削除されるアカウント（self-destructアカウント）の集合
- $A_{\mathbf{l}}$ : 生成されるログ（EVMコード内の実行時についたマーキング）の集合
- $A_{\mathbf{t}}$ : 変更されるアカウントの集合 / この中で空である($\mathtt{EMPTY}$)ものは削除される。
- $A_{\mathrm{r}}$ : EVMにおけるSSTORE命令で保存する値を「0」にした時に払い戻しがされる（ただしgasの埋め合わせをするだけ）。その払い戻し総額。

の意味。

空の副ステート$A^0$を以下で定義する。

$$
A^0 \equiv (\varnothing,(), \varnothing, 0)
$$

### 6.2 実行

７章と8章で定義される$\Lambda$、$\Omega$という関数を使って、ステートを更新する。$\Lambda$はコントラクト作成時の処理(=送金先指定なし)で、$\Omega$はコントラクト呼び出し時(&通常アドレス送金時)の処理である。

まずはじめにトランザクションそのものにかかる、基礎手数料$g_0$を定義する。

\begin{align}
g_0 \equiv {} & \sum_{i \in T_{\mathbf{i}}, T_{\mathbf{d}}} \begin{cases} G_{\mathrm{txdatazero}} & \text{if} \quad i = 0 \\ G_{\mathrm{txdatanonzero}} & \text{otherwise} \end{cases} \\
{} & + \begin{cases} G_{\mathrm{txcreate}} & \text{if} \quad T_{\mathrm{t}} = \varnothing \\ 0 & \text{otherwise} \end{cases} \\
{} & + G_{\mathrm{transaction}}
\end{align}

これは、

- トランザクションはコントラクト作成時の$T_{\mathbf{i}}$とアドレス呼出時の$T_{\mathbf{d}}$のどちらか一方が備わっているが、その中の1バイトにつき、内容が"00"であれば$G_\mathrm{txdatazero}$、それ以外であれば$G_\mathrm{txdatanonzero}$が課される。例えば、$T_{\mathbf{i}}$ないし$T_{\mathbf{d}}$が"00F711000000"であれば$4G_\mathrm{txdatazero} + 2G_\mathrm{txdatanonzero}$が課される。
- トランザクションに送り先がない=コントラクト作成の時は、 $G_{\mathrm{txcreate}}$が課される。
- いかなる場合でも必ず$G_{\mathrm{transaction}}$が課される。

と言う規則が適用されると言う意味である。$G$は様々な目的に応じたgasを定義している表であり、付録Gに記述されている。

単にアドレスに送金するだけであれば、$G_{\mathrm{transaction}} = 21000$が課される。

senderが持つ必要のある最低限の残高$v_0$を以下で定義する。

$$
v_0 \equiv T_{\mathrm{g}} T_{\mathrm{p}} + T_{\mathrm{v}}
$$

トランザクションの正当性は以下で表される。この条件をもって、ブロックにトランザクションを取り込む=ステートに影響を与えることができる。

$$
\begin{array}[t]{rcl}
S(T) & \neq & \varnothing \quad \wedge \\
\boldsymbol{\sigma}[S(T)] & \neq & \varnothing \quad \wedge \\
\linkdest{transaction_nonce}{}T_{\mathrm{n}} & = & \boldsymbol{\sigma}[S(T)]_{\mathrm{n}} \quad \wedge \\
g_0 & \leqslant & T_{\mathrm{g}} \quad \wedge \\
v_0 & \leqslant & \boldsymbol{\sigma}[S(T)]_{\mathrm{b}} \quad \wedge \\
T_{\mathrm{g}} & \leqslant & {B_{H}}_{\mathrm{l}} - \hyperlink{ell}{\ell}(B_{\mathbf{R}})_{\mathrm{u}}
\end{array}
$$

上から順に意味を説明すると、

- トランザクションのv, r, sから正しくsenderを復元できる
- 計算したsenderの指し示すアドレスが実際に存在する
- トランザクションのnonceはsenderのnonceと等しい
- $g_0$がトランザクションのgasLimit以下
- senderが最低でも$v_0$の分の残高を持っている
- ブロックの中で、今までに「実際に」消費したgasの量(gasLimitではなく、トランザクションを計算した結果のgas総量)と、現在処理しているトランザクションのgasLimitとの和($\ell(B_{\mathbf{R}})_{\mathrm{u}}+T_{\mathrm{g}}$)が、ブロックのgasLimit以下である。

$\ell$関数は配列の最後の値を参照する関数で、この場合は最後に処理されたトランザクションのレシートを参照している。

この後、

- gasLimit分の引き落としやnonceの変更をしたステート$\boldsymbol{\sigma}_0$
 - 処理: senderのnonceが1加算される
 - 処理: senderから$T_{\mathrm{g}} T_{\mathrm{p}}$分引き落とされる
- プログラムが実行された結果のステート$\boldsymbol{\sigma}_P$
 - 処理: $\Lambda$と$\Omega$によりEVMが実行される
- gasの差額調整後のステート$\boldsymbol{\sigma}^*$
 - 処理: gasの差額分がsenderに振込される
 - 処理: ブロックのマイナーに使用されたgasが振込される
- 最終ステート$\boldsymbol{\sigma}^{'}$
 - 処理: self-destructアカウントの削除
 - 処理: 空のアカウントの削除

の順に計算していく。

$\boldsymbol{\sigma}_0$では、senderからgasLimit分$T_{\mathrm{g}} T_{\mathrm{p}}$が引き落とされる。この時点では送金額$T_{\mathrm{v}}$は引き落とされない。さらにsenderのnonce(トランザクション送信カウント)が1加算される。

\begin{eqnarray}
\boldsymbol{\sigma}_0 & \equiv & \boldsymbol{\sigma} \quad \text{except:} \\
\boldsymbol{\sigma}_0[S(T)]_{\mathrm{b}} & \equiv & \boldsymbol{\sigma}[S(T)]_{\mathrm{b}} - T_{\mathrm{g}} T_{\mathrm{p}} \\
\boldsymbol{\sigma}_0[S(T)]_{\mathrm{n}} & \equiv & \boldsymbol{\sigma}[S(T)]_{\mathrm{n}} + 1
\end{eqnarray}

後々定義する$\Lambda, \Theta$を使ってEVMを実行する。$\Lambda$はコントラクト作成時、$\Theta$はコントラクト呼び出し時(と通常アカウントへの送金)の処理。$\Lambda, \Theta$の入力、出力の意味は以下のようになっている。

- $\Lambda$ : コントラクト作成時
 - 入力: 開始ステート, sender, トランザクション作成者, 使用できるgas, gasPrice, 送金額, 初期化EVMコード, 呼び出し深度, ステートへの書き込みができるか否か
 - 出力: 変更後のステート, 残りgas, 副ステート$A$, ステータスコード(失敗=0 成功=1), プログラムの出力結果
- $\Theta$ : コントラクト呼び出し時・通常アカウントへの送金時
 - 入力: 開始ステート, sender, トランザクション作成者, 使用できるgas, gasPrice, 実際の送金額, プログラムに通知される送金額, プログラムへの入力, 呼び出し深度, ステートへの書き込みができるか否か
 - 出力: $\Lambda$と同じ

Ethereumではコントラクトが別のコントラクトを呼びだすことができる。すなわち、$\Lambda$は再帰的に呼び出される可能性があるということである。そのために、

$\Lambda, \Theta$自体は5つの返値をもつが、その最初の4つのみを取得するという意味で$\Lambda_{4}, \Theta_{4}$となっている。

\begin{equation}
(\boldsymbol{\sigma}_{P}, g', A, z) \equiv \begin{cases}
\Lambda_{4}(\boldsymbol{\sigma}_0, S(T), T_{\mathrm{o}}, g, &\\ \quad\quad T_{\mathrm{p}}, T_{\mathrm{v}}, T_{\mathbf{i}}, 0, \top) & \text{if} \quad T_{\mathrm{t}} = \varnothing \\
\Theta_{4}(\boldsymbol{\sigma}_0, S(T), T_{\mathrm{o}}, T_{\mathrm{t}}, T_{\mathrm{t}}, &\\ \quad\quad g, T_{\mathrm{p}}, T_{\mathrm{v}}, T_{\mathrm{v}}, T_{\mathbf{d}}, 0, \top) & \text{otherwise}
\end{cases}
\end{equation}


\begin{equation}
g \equiv T_{\mathrm{g}} - g_0
\end{equation}

\begin{equation}
	A'_{r} \equiv A_{r} + \sum_{i \in A_{s}} R_{selfdestruct}
\end{equation}


\begin{equation}
g^* \equiv g' + \min \left\{ \Big\lfloor \dfrac{T_{\mathrm{g}} - g'}{2} \Big\rfloor, \hyperlink{refund_balance_defn_words_A__r}{A'_{\mathrm{r}}} \right\}
\end{equation}

\begin{eqnarray}
\boldsymbol{\sigma}^* & \equiv & \boldsymbol{\sigma}_{P} \quad \text{except} \\
\boldsymbol{\sigma}^*[S(T)]_{\mathrm{b}} & \equiv & \boldsymbol{\sigma}_{P}[S(T)]_{\mathrm{b}} + g^* T_{\mathrm{p}} \\
\boldsymbol{\sigma}^*[m]_{\mathrm{b}} & \equiv & \boldsymbol{\sigma}_{P}[m]_{\mathrm{b}} + (T_{\mathrm{g}} - g^*) T_{\mathrm{p}} \\
m & \equiv & {B_{H}}_{\mathrm{c}}
\end{eqnarray}

\begin{eqnarray}
\boldsymbol{\sigma}' & \equiv & \boldsymbol{\sigma}^* \quad \text{except} \\
\linkdest{self_destruct_list_A__s}{}\forall i \in A_{\mathbf{s}}: \boldsymbol{\sigma}'[i] & = & \varnothing \\
\linkdest{touched_A__t}{}\forall i \in A_{\mathbf{t}}: \boldsymbol{\sigma}'[i] & = & \varnothing \quad\text{if}\quad \mathtt{DEAD}(\boldsymbol{\sigma}^*\kern -2pt, i)
\end{eqnarray}

\begin{eqnarray}
\Upsilon^{\mathrm{g}}(\boldsymbol{\sigma}, T) & \equiv & T_{\mathrm{g}} - g^* \\
\Upsilon^{\mathbf{l}}(\boldsymbol{\sigma}, T) & \equiv & \hyperlink{tx_log_series_wordy_defn_A__l}{A_{\mathbf{l}}} \\
\Upsilon^{\mathrm{z}}(\boldsymbol{\sigma}, T) & \equiv & z
\end{eqnarray}

## 7. コントラクト作成

この章ではEthereum内でコントラクトアカウント（EVMプログラムがコードされたアカウント）が作成される仕組みについて記述する。



## 8. コントラクト呼び出し

この章ではEthereum内で作成されたコントラクトアカウントにデータを入力する仕組みについて記述する。


## 9. EVMコード実行

この章ではEVMプログラムを実行する仮想マシンの実装について記述する。


## 10. ブロックツリーからブロックチェーンに

Ethereumでは、「もっとも計算された」ブロックチェーンをノードが受け入れる仕組みになっている。



## 11. ブロックの最終処理

今までの話をまとめて、ブロック・トランザクションが正当なものか判別し、マイニング報酬を与える処理を記述する。




## 12. こんごのもくひょう





## 14. 付録（本編）

Yellow Paperは付録が本編です。

